In [ ]:
from typing import Dict
from typing import Union

import pandas as pd

from evidently import ColumnType
from evidently.model.widget import BaseWidgetInfo

from evidently.v2.datasets import DatasetColumn

from evidently.v2.datasets import Scorer
from evidently.v2.metrics import Metric
from evidently.v2.datasets import Dataset
from evidently.v2.metrics import MetricResult
from evidently.v2.metrics import SingleValue
from evidently.v2.metrics import SingleValueCheck
from evidently.v2.metrics.base import MetricId
from evidently.v2.metrics.min import min_metric
from evidently.v2.presets import PresetResult

In [ ]:
from typing import Optional


class TextLengthScorer(Scorer):
    def __init__(self, column_name: str, alias: Optional[str] = None):
        super().__init__(alias or f"{column_name}: Text Length")
        self._column_name = column_name

    def generate_data(self, dataset: "Dataset") -> Union[DatasetColumn, Dict[str, DatasetColumn]]:
        lengths = dataset.column(self._column_name).data.apply(len)
        return DatasetColumn(type=ColumnType.Numerical, data=lengths)


In [ ]:
class ToxicityScorer(Scorer):
    def __init__(self, column_name: str, alias: Optional[str] = None):
        super().__init__(alias or f"{column_name}: Toxicity")
        self._column_name = column_name
    
    def generate_data(self, dataset: "Dataset") -> Union[DatasetColumn, Dict[str, DatasetColumn]]:
        from evidently.descriptors import ToxicityLLMEval
        from evidently.options.base import Options

        feature = ToxicityLLMEval().feature(self._column_name)
        data = feature.generate_features(dataset.as_dataframe(), None, Options())
        return {
            col: DatasetColumn(type=feature.get_type(f"{feature.get_fingerprint()}.{col}"), data=data[col])
            for col in data.columns
        }

In [ ]:
def my_scorer(data: DatasetColumn) -> DatasetColumn:
    return DatasetColumn(type=ColumnType.Numerical, data=data.data)

def my_scorer2(dataset: Dataset) -> Union[DatasetColumn, Dict[str, DatasetColumn]]:
    return dataset.column("column_1")

In [ ]:
from evidently.v2.scorers import CustomColumnScorer
from evidently.v2.scorers import CustomScorer
from evidently.v2.scorers import TextLength

data = pd.DataFrame(data={"column_1": [1, 2, 3, 4, -1, 5], "column_2": ["a", "aa", "aaaa", "aaaaaaa", "a", "aa"]})

dataset = Dataset.from_pandas(
    data,
    data_definition=None,
    scorers=[
        TextLength("column_2", alias="column 2 length"),
        ToxicityScorer("column_2"),
        CustomColumnScorer("column_2", my_scorer, alias="column 2 custom function"),
        CustomScorer(my_scorer2, alias="global custom function"),
    ],
)

dataset.as_dataframe()

In [ ]:
from evidently.v2.checks.numerical_checks import le, ge
from typing import Optional
from typing import List
from plotly.express import line

# user-friendly interface
def max_metric(column_name: str, checks: Optional[List[SingleValueCheck]] = None, group_by: Optional[str] = None) -> Metric:
    return MaxMetric(column_name, checks)


# implementation
class MaxMetric(Metric[SingleValue]):
    _column_name: str
    
    def __init__(self, column_name: str, checks: Optional[List[SingleValueCheck]] = None):
        super().__init__(f"max:{column_name}")
        self._column_name = column_name
        self._checks = checks if checks is not None else [le(10), ge(6)]

    def calculate(self, current_data: Dataset, reference_data: Optional[Dataset]) -> SingleValue:
        x = current_data.column(self._column_name).data
        value = x.max()
        result = SingleValue(value=value)
        figure = line(x)
        figure.add_hrect(6, 10)
        #result.set_widget([plotly_figure(title=self.display_name(), figure=figure)])
        return result

    def display_name(self) -> str:
        return f"Max value for {self._column_name}"


result = max_metric("column_1", checks=[]).call(dataset, None)
result

In [ ]:
from evidently.v2.metrics.group_by import GroupBy
from evidently.v2.metrics.base import render_results
from evidently.v2.report import Context

context = Context()

context.init_dataset(dataset, None)

metrics = GroupBy(max_metric("column 2 length"), "column_1").generate_metrics(context)

results = [metric.call(dataset, None) for metric in metrics]

render_results(results)

In [ ]:
results[0].value

In [ ]:
from evidently.v2.presets import MetricPreset

class ColumnSummary(MetricPreset):
    def __init__(self, column: str):
        self._column = column

    def metrics(self) -> List[Metric]:
        return [
            min_metric(self._column),
            max_metric(self._column),
        ]
    
    def calculate(self, metric_results: Dict[MetricId, MetricResult]) -> PresetResult:
        return PresetResult(widget=[
            *metric_results[min_metric(self._column).id].widget,
            *metric_results[max_metric(self._column).id].widget,
        ])


In [ ]:
ColumnSummary("column_1").call(context)

In [ ]:
from evidently.v2.report import Report

report = Report([min_metric("column_1"), max_metric("column_1")])
snapshot = report.run(dataset, None)
snapshot